Необходимо написать алгоритм, который по списку любимых жанров пользователя выдает фильмы этих жанров с максимальным рейтингом. Берем фильмы, содержащие более 100 оценок.
Список любимых жанров пользователя задается в листе user_genres


Итак, у нас есть названия фильмов и их жанры в файле movies.csv. И данные по рейтингам фильмов в файле ratings.csv.
Начнем с загрузки данных в Pandas:

In [1]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')

In [2]:
min_ratings_count = 100 #зададим минимальнное количество оценок, равное 100

movies_count = ratings.groupby('movieId').count().reset_index() # датафрэйм сгруппированный по ID с указанием кол-ва записей
movies_count.head() # OK!

,movieId,userId,rating,timestamp
0,1,247,247,247
1,2,107,107,107
2,3,59,59,59
3,4,13,13,13
4,5,56,56,56


Считаем средний рейтинг для фильмов, которые содержат 100 и более оценок:

In [3]:
movies_with_n_ratings = movies_count[ movies_count['rating'] >= min_ratings_count ] # датафрейм с количеством оценок >=100
movies_with_n_ratings.head() # OK!

,movieId,userId,rating,timestamp
0,1,247,247,247
1,2,107,107,107
5,6,104,104,104
9,10,122,122,122
24,25,101,101,101


In [4]:
list_of_movies_with_n_ratings = movies_with_n_ratings['movieId'].tolist() # создадим лист из ID фильмов >=100
# list_of_movies_with_n_ratings # OK!


In [5]:
ratings = ratings[ ratings['movieId'].isin(list_of_movies_with_n_ratings) ] 
# создадим новый ДФ, в котором movieId есть в созданном ранее Листе
ratings.head()

,userId,movieId,rating,timestamp
20,2,10,4.0,835355493
22,2,39,5.0,835355604
23,2,47,4.0,835355552
24,2,50,4.0,835355586
27,2,110,4.0,835355532


Считаем средний рейтинг для каждого фильма

In [6]:
ratings = ratings.groupby('movieId').mean().reset_index()
ratings.head() # В данном ДФ приведен средний рейтинг фильмов со 100 и болле оценками

,movieId,userId,rating,timestamp
0,1,338.558704,3.872470,1.103116e+09
1,2,318.906542,3.401869,1.069321e+09
2,6,334.096154,3.884615,1.001189e+09
3,10,316.131148,3.450820,9.977456e+08
4,25,348.564356,3.742574,9.790063e+08


### Добавляем названия и жанры
Загружаем данные с жанрами:

In [7]:
movies = pd.read_csv('movies.csv')

Добавим к каждому фильму информацию о жанрах

In [8]:
joined = ratings.merge(movies, on = 'movieId', how = 'left')
joined.head()

,movieId,userId,rating,timestamp,title,genres
0,1,338.558704,3.872470,1.103116e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,318.906542,3.401869,1.069321e+09,Jumanji (1995),Adventure|Children|Fantasy
2,6,334.096154,3.884615,1.001189e+09,Heat (1995),Action|Crime|Thriller
3,10,316.131148,3.450820,9.977456e+08,GoldenEye (1995),Action|Adventure|Thriller
4,25,348.564356,3.742574,9.790063e+08,Leaving Las Vegas (1995),Drama|Romance


Проверим на всякий случай, что число строк после объединения не изменилось:

In [9]:
len(ratings) == len(joined)

True

### Задание
Теперь необходимо сделать следующее:

1. Написать функцию, которая для каждой строки датафрейма joined определяет входит ли ее жанр в лист user_genres. Например, добавить столбец user_genre со значениями True (у фильма есть жанр из листа user_genres) и False.

2. Отфильтровать датафрейм joined по значению user_genre = True

3. Отсортировать датафрейм joined по уменьшению среднего рейтинга фильма

In [10]:
user_genres = ['Comedy', 'Romance']

def genres_matching(row):
    # делим набор жанров из столбца 'genres' на отдельные жанры
    genres_list_from_row = row['genres'].split('|')
    # проверяем есть ли в получившемся листе жанр из листа user_genres
    for genre in genres_list_from_row:
        if genre in user_genres:
            return True
    return False

Теперь осталось сформировать новый столбец в датафрейме movies и проверить работу функции. Для этого воспользуемся методом apply, в котором укажем используемую функцию. Параметр axis = 1 указывает на то, что в функцию передается строка. Если поставить axis = 0, то будет обрабатываться столбец.

In [11]:
joined['user_genre'] = joined.apply(genres_matching, axis = 1)
joined.head()

,movieId,userId,rating,timestamp,title,genres,user_genre
0,1,338.558704,3.872470,1.103116e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,True
1,2,318.906542,3.401869,1.069321e+09,Jumanji (1995),Adventure|Children|Fantasy,False
2,6,334.096154,3.884615,1.001189e+09,Heat (1995),Action|Crime|Thriller,False
3,10,316.131148,3.450820,9.977456e+08,GoldenEye (1995),Action|Adventure|Thriller,False
4,25,348.564356,3.742574,9.790063e+08,Leaving Las Vegas (1995),Drama|Romance,True


In [12]:
joined.sort_values(by=['user_genre'], ascending=False).head()

,movieId,userId,rating,timestamp,title,genres,user_genre
0,1,338.558704,3.872470,1.103116e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,True
63,912,347.273504,4.235043,1.099906e+09,Casablanca (1942),Drama|Romance,True
114,2716,335.482993,3.761905,1.130762e+09,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi,True
60,778,321.733871,4.141129,1.149092e+09,Trainspotting (1996),Comedy|Crime|Drama,True
59,750,338.266667,4.209524,1.126820e+09,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,True


In [13]:
joined.sort_values(by=['user_genre','rating'], ascending=False).head()

,movieId,userId,rating,timestamp,title,genres,user_genre
55,608,348.142857,4.256696,1.060985e+09,Fargo (1996),Comedy|Crime|Drama|Thriller,True
25,296,336.910494,4.256173,1.088122e+09,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,True
118,2858,322.200000,4.236364,1.132165e+09,American Beauty (1999),Drama|Romance,True
63,912,347.273504,4.235043,1.099906e+09,Casablanca (1942),Drama|Romance,True
70,1136,347.241379,4.224138,1.149455e+09,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,True
